In [48]:
import sympy
from sympy import symbols, Function, Eq, simplify, solve, factor
from sympy.vector import CoordSys3D

In [40]:
p0,p1,p2,p3 = symbols("p_0:4", vector=2)
tq = symbols("t_q", real=True)
lerp = lambda a,b,t: a+(b-a)*t

h0=lerp(p0,p1,tq)
h1=lerp(p1,p2,tq)
h2=lerp(p2,p3,tq)
q0=lerp(h0,h1,tq)
q1=lerp(h1,h2,tq)

q = simplify(lerp(q0,q1,tq))
q

-p_0*t_q**3 + 3*p_0*t_q**2 - 3*p_0*t_q + p_0 + 3*p_1*t_q**3 - 6*p_1*t_q**2 + 3*p_1*t_q - 3*p_2*t_q**3 + 3*p_2*t_q**2 + p_3*t_q**3

In [60]:
from sympy import symbols, Matrix, simplify

# Define scalar parameter
tq = symbols("t_q", real=True)

# Define control points as 2D vectors
p0 = Matrix(symbols("p0_x p0_y", real=True))
p1 = Matrix(symbols("p1_x p1_y", real=True))
p2 = Matrix(symbols("p2_x p2_y", real=True))
p3 = Matrix(symbols("p3_x p3_y", real=True))

# Define linear interpolation function for vectors
lerp = lambda a, b, t: a + (b - a) * t

# Compute intermediate points
h0 = lerp(p0, p1, tq)
h1 = lerp(p1, p2, tq)
h2 = lerp(p2, p3, tq)
q0 = lerp(h0, h1, tq)
q1 = lerp(h1, h2, tq)

# Compute final Bézier curve point
q = simplify(lerp(q0, q1, tq))

q

Matrix([
[-p0_x*t_q**3 + 3*p0_x*t_q**2 - 3*p0_x*t_q + p0_x + 3*p1_x*t_q**3 - 6*p1_x*t_q**2 + 3*p1_x*t_q - 3*p2_x*t_q**3 + 3*p2_x*t_q**2 + p3_x*t_q**3],
[-p0_y*t_q**3 + 3*p0_y*t_q**2 - 3*p0_y*t_q + p0_y + 3*p1_y*t_q**3 - 6*p1_y*t_q**2 + 3*p1_y*t_q - 3*p2_y*t_q**3 + 3*p2_y*t_q**2 + p3_y*t_q**3]])

In [61]:
tab = symbols("t_ab", real=True)
pa = Matrix(symbols("pa_x pa_y", real=True))
pb = Matrix(symbols("pb_x pb_y", real=True))

line = lerp(pa,pb,tab)
line

Matrix([
[pa_x + t_ab*(-pa_x + pb_x)],
[pa_y + t_ab*(-pa_y + pb_y)]])

In [62]:
problem = Eq(line,q)
problem

Eq(Matrix([
[pa_x + t_ab*(-pa_x + pb_x)],
[pa_y + t_ab*(-pa_y + pb_y)]]), Matrix([
[-p0_x*t_q**3 + 3*p0_x*t_q**2 - 3*p0_x*t_q + p0_x + 3*p1_x*t_q**3 - 6*p1_x*t_q**2 + 3*p1_x*t_q - 3*p2_x*t_q**3 + 3*p2_x*t_q**2 + p3_x*t_q**3],
[-p0_y*t_q**3 + 3*p0_y*t_q**2 - 3*p0_y*t_q + p0_y + 3*p1_y*t_q**3 - 6*p1_y*t_q**2 + 3*p1_y*t_q - 3*p2_y*t_q**3 + 3*p2_y*t_q**2 + p3_y*t_q**3]]))

In [66]:
solve(problem, tab)

[]

In [67]:
import numpy as np
from scipy.optimize import root

# Define the line segment
def line(t_line, pa, pb):
    return (1 - t_line) * np.array(pa) + t_line * np.array(pb)

# Define the cubic Bézier curve
def bezier(t_curve, p0, p1, p2, p3):
    h0 = (1 - t_curve) * np.array(p0) + t_curve * np.array(p1)
    h1 = (1 - t_curve) * np.array(p1) + t_curve * np.array(p2)
    h2 = (1 - t_curve) * np.array(p2) + t_curve * np.array(p3)
    q0 = (1 - t_curve) * h0 + t_curve * h1
    q1 = (1 - t_curve) * h1 + t_curve * h2
    return (1 - t_curve) * q0 + t_curve * q1

# Define the system of equations to solve
def intersection_system(vars, pa, pb, p0, p1, p2, p3):
    t_line, t_curve = vars
    L = line(t_line, pa, pb)
    B = bezier(t_curve, p0, p1, p2, p3)
    return L - B  # Return the difference (target is zero)

# Example points for the line and Bézier curve
pa = [0, 0]
pb = [1, 1]
p0 = [0, 0]
p1 = [0.5, 1]
p2 = [1, 1]
p3 = [1, 0]

# Initial guess for t_line and t_curve
initial_guess = [0.5, 0.5]

# Solve the system numerically
solution = root(intersection_system, initial_guess, args=(pa, pb, p0, p1, p2, p3))

if solution.success:
    t_line, t_curve = solution.x
    print(f"Intersection found at:")
    print(f"Line parameter t_line = {t_line}")
    print(f"Bezier parameter t_curve = {t_curve}")
    print(f"Intersection point: {line(t_line, pa, pb)}")
else:
    print("No intersection found.")

Intersection found at:
Line parameter t_line = 0.7423461417479416
Bezier parameter t_curve = 0.550510257216999
Intersection point: [0.74234614 0.74234614]
